In [5]:
from arcpy.sa import *
import pandas as pd
import os
from pathlib import Path

In [2]:
cities = pd.read_csv('centroids.csv').city

In [3]:
floods = ['fu', 'pu']

### WSF

In [13]:
for city in cities:
    city1 = city.lower().replace(' ', '_')
    for flood in floods:
        TabulateArea(city + '/data/class_all_' + flood + '.tif', 'Value',
                     Int(city + '/data/' + city1 + '_WSFevolution_utm.tif'), 'Value',
                     city1.replace('-', '_') + '_' + flood)

In [20]:
for city in cities:
    city1 = city.lower().replace(' ', '_')
    for flood in floods:
        arcpy.conversion.TableToTable(city1.replace('-', '_') + '_' + flood, city + '/data', city1.replace('-', '_') + '_' + flood + '.csv')

### Built up projection

In [16]:
ssps = ['1', '2', '3']

In [17]:
years = ['2050', '2100']

In [18]:
arcpy.env.cellSize = 'MINOF'

In [19]:
for city in cities:
    city1 = city.lower().replace(' ', '_')
    city2 = city1.replace('-', '_')
    for flood in floods:
        for ssp in ssps:
            for year in years:
                ZonalStatisticsAsTable(city + '/data/class_all_' + flood + '.tif', 
                                       "Value", city + '/data/' + city1 + '_bu_ssp' + ssp + '_' + year + '.tif', 
                                       city2 + '_ssp' + ssp + '_' + flood + '_' + year, "DATA", "SUM")

In [21]:
for city in cities:
    city1 = city.lower().replace(' ', '_')
    city2 = city1.replace('-', '_')
    for flood in floods:
        for ssp in ssps:
            for year in years:
                arcpy.conversion.TableToTable(city2 + '_ssp' + ssp + '_' + flood + '_' + year, city + '/data', city2 + '_ssp' + ssp + '_' + flood + '_' + year + '.csv')

### SLR

In [23]:
coastal_cities = []
for city in cities:
    city1 = city.lower().replace(' ', '_')
    if os.path.exists(city + '/data/' + city1 + '_slr_RL10_2100.tif'):
        coastal_cities.append(city)
coastal_cities

['Matadi']

In [24]:
years = ['2050', '2100']
slr_types = ['', '_RL10']

In [26]:
for city in coastal_cities:
    city1 = city.lower().replace(' ', '_')
    city2 = city1.replace('-', '_')
    for year in years:
        for slr in slr_types:
            TabulateArea(Int(city + '/data/' + city1 + '_slr' + slr + '_' + year + '.tif'), 'Value',
                         Int(city + '/data/' + city1 + '_WSFevolution_utm.tif'), 'Value',
                         city2 + '_slr' + slr + '_' + year)

In [27]:
for city in coastal_cities:
    city1 = city.lower().replace(' ', '_')
    city2 = city1.replace('-', '_')
    for year in years:
        for slr in slr_types:
            arcpy.conversion.TableToTable(city2 + '_slr' + slr + '_' + year, city + '/data', city2 + '_slr' + slr + '_' + year + '.csv')

### Drought

Get monthly total water storage anomaly values for each city during 2011-2020.

In [28]:
raster_list = ['twsan']
year_list = range(2011, 2021)
month_list = ['01', '02', '03', 
              '04', '05', '06', 
              '07', '08', '09', 
              '10', '11', '12']

In [29]:
for raster in raster_list:
    for year in year_list:
        for month in month_list:
            Sample('F:/World Bank/City Scan/data/Drought/twsan_m_wld_20200101_20201201_m/'+raster+'_m_wld_'+str(year)+month+'01_m.tif',  # update file path as needed
                   'shapefile/centroids.shp', raster+'_'+str(year)+month, 'CUBIC')

In [30]:
for raster in raster_list:
    for year in year_list:
        for month in month_list:
            arcpy.conversion.TableToTable(raster+'_'+str(year)+month, 
                                          'stats/drought', 
                                          raster+'_'+str(year)+month+'.csv')